# SingleLensFitter - Complete Documentation and Usage Guide

This notebook provides comprehensive documentation for the `SingleLensFitter` class, which is used for modeling and analyzing single-lens gravitational microlensing events.

## Table of Contents
1. [Introduction](#introduction)
2. [Basic Usage](#basic-usage)
3. [Model Configuration Options](#model-configuration)
4. [Advanced Features](#advanced-features)
5. [Complete Example with All Features](#complete-example)
6. [Template for Your Own Data](#your-data-template)

---

## 1. Introduction <a name="introduction"></a>

The `SingleLensFitter` class fits gravitational microlensing light curves using:
- **Point-Source, Point-Lens (PSPL)** model as baseline
- Optional **finite source effects** with limb darkening
- **Gaussian Process** models for correlated noise
- **Mixture models** for outlier rejection
- **MCMC sampling** using `emcee` for parameter estimation

### Required Parameters
The fundamental microlensing parameters are:
- **u₀**: Impact parameter (closest approach in Einstein radius units)
- **t₀**: Time of closest approach (HJD - 2450000)
- **tₑ**: Einstein crossing time (days)

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from SingleLensFitter import SingleLensFitter

# Set up matplotlib for inline display
%matplotlib inline

## 2. Basic Usage <a name="basic-usage"></a>

### 2.1 Prepare Your Data

Data must be organized as a dictionary where:
- **Keys**: Dataset names (e.g., observatory names)
- **Values**: Tuples of `(time, flux, flux_error)` as numpy arrays

In [ ]:
# Generate synthetic microlensing data for demonstration
def generate_synthetic_event(t0=7500.0, u0=0.1, tE=30.0, F_source=1000.0, F_blend=500.0, noise_level=20.0, n_points=500):
    """
    Generate synthetic microlensing light curve.
    
    Parameters:
    -----------
    t0 : float
        Time of closest approach
    u0 : float
        Impact parameter
    tE : float
        Einstein crossing time
    F_source : float
        Source flux
    F_blend : float
        Blend flux
    noise_level : float
        Standard deviation of Gaussian noise
    n_points : int
        Number of data points
    """
    time = np.linspace(t0 - 2 * tE, t0 + 2 * tE, n_points)
    tau = (time - t0) / tE
    u = np.sqrt(u0**2 + tau**2)
    magnification = (u**2 + 2) / (u * np.sqrt(u**2 + 4))
    
    flux = F_source * magnification + F_blend
    flux += np.random.normal(0, noise_level, len(time))
    err = np.ones_like(flux) * noise_level
    
    return time, flux, err

# Generate data for a single observatory
time, flux, err = generate_synthetic_event()
data_dict = {'Observatory1': (time, flux, err)}

# Visualize the data
plt.figure(figsize=(10, 6))
plt.errorbar(time, flux, yerr=err, fmt='.', alpha=0.6, label='Observatory1')
plt.xlabel('Time (HJD - 2450000)')
plt.ylabel('Flux')
plt.title('Synthetic Microlensing Event')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 2.2 Initialize the Fitter

Provide initial parameter guesses: `[u0, t0, tE]`

In [ ]:
# Set initial parameter guesses
initial_params = np.array([0.15, 7505.0, 25.0])  # [u0, t0, tE]

# Initialize the fitter
fitter = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params,
    ZP=28.0  # Magnitude zero-point for plotting
)

print(f"Initialized fitter with {fitter.ndim} parameters")
print(f"Parameter labels: {fitter.parameter_labels}")

### 2.3 Run a Simple Fit

Use optimization to find the best-fit parameters quickly.

In [ ]:
# Quick optimization (useful for finding starting point)
fitter.fit(method='Nelder-Mead')

print(f"\nOptimized parameters:")
print(f"u0 = {fitter.p[0]:.4f}")
print(f"t0 = {fitter.p[1]:.4f}")
print(f"tE = {fitter.p[2]:.4f}")

## 3. Model Configuration Options <a name="model-configuration"></a>

### 3.1 Finite Source Effects

Add finite source effects when the source size is comparable to the Einstein radius.

In [ ]:
# Create a new fitter instance
fitter_fs = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add finite source effects
fitter_fs.add_finite_source(lrho=-3.0)  # log10(rho), where rho = R_*/R_E

print(f"Number of parameters with finite source: {fitter_fs.ndim}")
print(f"Parameter labels: {fitter_fs.parameter_labels}")

### 3.2 Limb Darkening

Add linear limb darkening (automatically includes finite source effects).

In [ ]:
# Create a new fitter instance
fitter_ld = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add limb darkening (includes finite source automatically)
fitter_ld.add_limb_darkening(
    gamma=0.5,   # Linear limb-darkening coefficient (0-1)
    lrho=-2.5    # log10(source radius)
)

print(f"Number of parameters with limb darkening: {fitter_ld.ndim}")
print(f"Parameter labels: {fitter_ld.parameter_labels}")

### 3.3 Source Variability

Model sinusoidal variability in the source star.

In [ ]:
# Create a new fitter instance
fitter_sv = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add source variability: F_source * (1 + K*sin(omega*t + phi))
fitter_sv.add_source_variability(
    params=(0.001, np.pi, 0.0)  # (K, omega, phi)
)

print(f"Number of parameters with source variability: {fitter_sv.ndim}")
print(f"Parameter labels: {fitter_sv.parameter_labels}")

### 3.4 Blend Variability

Model sinusoidal variability in the blend flux.

In [ ]:
# Create a new fitter instance
fitter_bv = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add blend variability: F_blend * (1 + K*sin(omega*t + phi))
fitter_bv.add_blend_variability(
    params=(0.001, np.pi, 0.0)  # (K, omega, phi)
)

print(f"Number of parameters with blend variability: {fitter_bv.ndim}")
print(f"Parameter labels: {fitter_bv.parameter_labels}")

### 3.5 Gaussian Process Model

Model correlated noise using a Gaussian Process with an exponential kernel.

In [ ]:
# Example: Using fast linear fit for quick testing

# Generate simple test data
time_fast, flux_fast, err_fast = generate_synthetic_event(
    t0=7500, u0=0.2, tE=25, noise_level=15, n_points=1000
)
data_fast = {'QuickTest': (time_fast, flux_fast, err_fast)}

# Initialize with simple PSPL model
fitter_fast = SingleLensFitter(
    data=data_fast,
    initial_parameters=np.array([0.25, 7502, 23])
)

# Enable fast linear fit
fitter_fast.use_fast_linear_fit = True

# Configure for quick test
fitter_fast.nwalkers = 30
fitter_fast.nsteps_production = 200
fitter_fast.plotprefix = 'fast_fit_test'

print("Fast linear fit enabled: ", fitter_fast.use_fast_linear_fit)
print("This will use lightweight weighted least squares instead of full matrix inversion.")
print("\nRunning quick optimization...")

# Quick optimization
fitter_fast.fit(method='Nelder-Mead')

print(f"\nOptimized parameters:")
print(f"  u0 = {fitter_fast.p[0]:.4f}")
print(f"  t0 = {fitter_fast.p[1]:.4f}")
print(f"  tE = {fitter_fast.p[2]:.4f}")

# Note: For MCMC sampling, the fast fit will be used automatically
# fitter_fast.sample(optimize_first=False)

### 3.6 Eigen Lightcurves (Detrending Vectors)

Eigen lightcurves are pre-computed basis vectors used to model and remove systematic effects in the data. This is particularly useful for removing trends caused by:
- Atmospheric variations
- Instrumental drifts
- Seeing variations
- Background contamination
- Any other systematic effect that can be modeled linearly

**How it works**:
1. Create eigen vectors (basis functions) from your data or external sources
2. The fitter linearly combines these vectors to model systematic trends
3. The trend is fitted simultaneously with the microlensing signal
4. Different observatories can have different sets of eigen lightcurves

**Common sources of eigen lightcurves**:
- Principal Component Analysis (PCA) of reference stars
- Time-dependent PSF models
- Known instrumental systematics
- Environmental monitors (temperature, humidity, etc.)

**Mathematical model**:
For each observatory, the flux is modeled as:

$$F(t) = F_s \cdot A(t) + F_b + \sum_{i=1}^{N_{eigen}} c_i \cdot E_i(t)$$

Where:
- $F_s$ = source flux
- $A(t)$ = magnification
- $F_b$ = blend flux
- $c_i$ = eigen lightcurve coefficients (fitted parameters)
- $E_i(t)$ = eigen lightcurve basis vectors (provided by user)

### Creating Eigen Lightcurves from Your Data

**Method 1: Principal Component Analysis (PCA) from Reference Stars**

If you have reference stars (non-variable stars in the same field), you can use PCA to extract systematic trends:

```python
from sklearn.decomposition import PCA

# Assume you have reference star light curves: ref_stars (n_stars × n_times)
# Each row is a different reference star, each column is a time point

# Subtract the mean from each reference star
ref_stars_centered = ref_stars - np.mean(ref_stars, axis=1, keepdims=True)

# Perform PCA
pca = PCA(n_components=3)  # Extract 3 principal components
pca.fit(ref_stars_centered.T)  # Fit on transposed data

# Get the principal components (eigen lightcurves)
eigen_lcs = pca.components_  # Shape: (3, n_times)

# These eigen lightcurves capture the dominant systematic variations
```

**Method 2: Known Instrumental Effects**

If you know specific instrumental or environmental effects:

```python
# Example: Temperature-dependent drift
temperature_curve = temperature_data  # From observatory logs
eigen_lc_temp = (temperature_curve - np.mean(temperature_curve)) / np.std(temperature_curve)

# Example: Airmass correction
airmass_curve = compute_airmass(time, target_coords, observatory_location)
eigen_lc_airmass = airmass_curve - np.mean(airmass_curve)

# Combine into eigen lightcurve array
eigen_lcs = np.vstack([eigen_lc_temp, eigen_lc_airmass])
```

**Method 3: Polynomial Detrending**

For simple time-dependent trends:

```python
# Create polynomial basis functions
time_normalized = (time - np.mean(time)) / np.std(time)
eigen_lc_linear = time_normalized
eigen_lc_quadratic = time_normalized ** 2
eigen_lc_cubic = time_normalized ** 3

eigen_lcs = np.vstack([eigen_lc_linear, eigen_lc_quadratic, eigen_lc_cubic])
```

**Important**: Always normalize your eigen lightcurves to have zero mean and unit variance for numerical stability.

In [ ]:
# Example: Using eigen lightcurves for detrending

# Generate data with a systematic trend
time_trend, flux_trend, err_trend = generate_synthetic_event(
    t0=7500, u0=0.15, tE=28, F_source=1000, F_blend=500, 
    noise_level=18, n_points=500
)

# Add a systematic trend (e.g., instrumental drift)
# This could be atmospheric effects, temperature drift, etc.
systematic_trend_1 = 50 * np.sin(2 * np.pi * time_trend / 20)  # ~20-day oscillation
systematic_trend_2 = 30 * np.exp(-(time_trend - 7500)**2 / (2 * 50**2))  # Gaussian bump

# Add systematics to the flux
flux_with_systematics = flux_trend + systematic_trend_1 + systematic_trend_2

# Create eigen lightcurves (basis vectors for detrending)
# In practice, these would come from PCA of reference stars or other analysis
# Here we'll create them to match our known systematics
eigen_lc_1 = np.sin(2 * np.pi * time_trend / 20)  # Sinusoidal component
eigen_lc_2 = np.exp(-(time_trend - 7500)**2 / (2 * 50**2))  # Gaussian component
eigen_lc_3 = np.ones_like(time_trend)  # Constant offset (optional)

# Stack into an array: shape is (n_eigen_vectors, n_data_points)
eigen_lightcurves_array = np.vstack([eigen_lc_1, eigen_lc_2, eigen_lc_3])

print(f"Created {eigen_lightcurves_array.shape[0]} eigen lightcurves")
print(f"Each has {eigen_lightcurves_array.shape[1]} data points")

# Create data dictionary
data_with_eigen = {'Observatory_A': (time_trend, flux_with_systematics, err_trend)}

# Create eigen lightcurves dictionary
# Keys must match the data dictionary keys
eigen_lcs_dict = {'Observatory_A': eigen_lightcurves_array}

# Initialize fitter WITH eigen lightcurves
fitter_eigen = SingleLensFitter(
    data=data_with_eigen,
    initial_parameters=np.array([0.18, 7502, 26]),
    eigen_lightcurves=eigen_lcs_dict  # Pass eigen lightcurves here
)

print(f"\nFitter initialized with eigen lightcurves for detrending")
print(f"Number of microlensing parameters: 3 (u0, t0, tE)")
print(f"Number of linear parameters per observatory: 2 (Fs, Fb) + {eigen_lightcurves_array.shape[0]} (eigen coeffs)")
print(f"The eigen coefficients will be marginalized over during fitting")

# Configure and optimize
fitter_eigen.plotprefix = 'eigen_detrend_test'
fitter_eigen.nwalkers = 40
fitter_eigen.nsteps_production = 300

print("\nRunning optimization...")
fitter_eigen.fit(method='Nelder-Mead')

print(f"\nOptimized microlensing parameters:")
print(f"  u0 = {fitter_eigen.p[0]:.4f}")
print(f"  t0 = {fitter_eigen.p[1]:.4f}")
print(f"  tE = {fitter_eigen.p[2]:.4f}")

print("\nThe systematic trends have been removed by fitting the eigen lightcurves!")
print("The eigen coefficients are automatically marginalized over in the likelihood.")

### 3.7 Fast Linear Fit Optimization

For simple PSPL models without advanced features, you can enable a lightweight linear fitting routine that is faster than the full matrix inversion method.

**When to use `use_fast_linear_fit`**:
- ✅ Simple PSPL model (only u₀, t₀, tₑ parameters)
- ✅ Blended model (fitting both source and blend flux)
- ✅ Quick testing or exploratory analysis
- ✅ Large datasets where speed is critical

**When NOT to use it**:
- ❌ Gaussian Process models
- ❌ Eigen lightcurves (detrending)
- ❌ Source or blend variability
- ❌ Mixture models
- ❌ Non-blended models

**Performance gain**: Typically 2-5× faster for simple models with large datasets.

**Multi-Observatory Eigen Lightcurves**:

Different observatories can have different systematic effects, so you can provide different eigen lightcurves for each:

```python
# Example structure for multi-observatory eigen lightcurves
eigen_lcs_multi = {
    'OGLE': eigen_array_ogle,      # shape: (n_eigen_ogle, n_points_ogle)
    'MOA': eigen_array_moa,         # shape: (n_eigen_moa, n_points_moa)
    'KMTNet': eigen_array_kmtnet    # shape: (n_eigen_kmtnet, n_points_kmtnet)
}

# Not all observatories need eigen lightcurves
eigen_lcs_partial = {
    'OGLE': eigen_array_ogle,  # OGLE has systematics to remove
    # MOA and KMTNet don't have eigen lightcurves - will use standard fitting
}
```

**Best Practices**:
1. **Orthogonalize**: Use PCA or Gram-Schmidt to create orthogonal basis vectors
2. **Normalization**: Normalize eigen vectors to have similar amplitudes
3. **Number of vectors**: Use 2-5 eigen lightcurves; too many can cause overfitting
4. **Validation**: Test with simulated data to ensure systematics are properly removed
5. **Physical meaning**: If possible, relate eigen vectors to known physical effects

**Technical Notes**:
- Eigen coefficients are treated as **nuisance parameters** and are marginalized over analytically
- This is computationally efficient - no MCMC sampling needed for eigen coefficients
- The marginalization is done during the linear fit using matrix algebra
- Eigen lightcurves increase the size of the design matrix but not the MCMC dimensionality
- Incompatible with `use_fast_linear_fit` (requires full matrix inversion)

### Implementation Details: Fast Linear Fit vs Full Linear Fit

**Understanding the Two Approaches**:

The `SingleLensFitter` has two methods for fitting linear parameters (source flux, blend flux, eigen coefficients):

#### 1. Full Linear Fit (`linear_fit_full`)
- **Method**: Cholesky decomposition with full covariance matrix inversion
- **Handles**: All features (GP, eigen lightcurves, variability, mixture models)
- **Mathematics**: 
  - Constructs design matrix **A** (magnification, eigen vectors, etc.)
  - Builds covariance matrix **C** (including GP correlations if used)
  - Solves: **S** = **A**ᵀ **C**⁻¹ **A**, **b** = **A**ᵀ **C**⁻¹ **y**
  - Parameters: **θ** = **S**⁻¹ **b**
  - Can marginalize over linear parameters analytically
- **Cost**: O(N³) for matrix inversion, where N = number of data points

#### 2. Fast Linear Fit (`linear_fit_fast`)
- **Method**: Weighted least squares without full matrix construction
- **Handles**: Simple PSPL only (no GP, no eigen lightcurves, no variability)
- **Mathematics**:
  - Design matrix: **A** = [**1**, **mag**]ᵀ (only constant and magnification)
  - Diagonal weights: **w** = 1/σ²
  - Solves: **S** = **A** diag(**w**) **A**ᵀ, **b** = **A** diag(**w**) **y**
  - Parameters: [F_blend, F_source] = **S**⁻¹ **b**
- **Cost**: O(N) - no full matrix inversion, just a 2×2 system
- **Speedup**: 2-5× faster for large datasets

**When is Fast Fit Used?**

The code automatically selects the fast fit when:
```python
can_use_fast_fit = (
    self.use_fast_linear_fit and                    # User enabled it
    not self.use_gaussian_process_model and         # No GP
    (self.eigen_lightcurves is None or              # No eigen lightcurves
     data_key not in self.eigen_lightcurves) and
    not self.use_source_variability and             # No source variability
    not self.use_blend_variability and              # No blend variability
    not self.use_mixture_model and                  # No mixture model
    self.fit_blended                                # Must fit both Fs and Fb
)
```

**Example from Source Code** (SingleLensFitter.py):
```python
def linear_fit_fast(self, data_key, mag):
    """Lightweight linear fit for simple models."""
    t, y, yerr = self.data[data_key]
    w = 1.0 / (yerr ** 2)
    A = np.vstack((np.ones_like(mag), mag))
    Aw = A * w
    Sw = Aw @ A.T                    # 2×2 matrix
    bw = Aw @ y
    a = np.linalg.solve(Sw, bw)      # Fast 2×2 solve
    model = a[0] + a[1] * mag
    chi2 = np.sum(((y - model) / yerr) ** 2)
    return a, -0.5 * chi2
```

**Recommendation**: Use fast fit for quick testing, but use full fit for production runs with complex models.

In [ ]:
# Create a new fitter instance
fitter_gp = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add Gaussian Process model
fitter_gp.add_gaussian_process_model(
    common=True  # Use same GP parameters for all datasets
)

print(f"Number of parameters with GP model: {fitter_gp.ndim}")
print(f"Parameter labels: {fitter_gp.parameter_labels}")
print(f"\nGP parameters: ln_a (log amplitude), ln_tau (log timescale)")

### 3.8 Mixture Model for Outlier Rejection

Add a mixture model to handle outliers robustly.

In [ ]:
# Create a new fitter instance
fitter_mm = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params
)

# Add mixture model
fitter_mm.add_mixture_model()

print(f"Number of parameters with mixture model: {fitter_mm.ndim}")
print(f"Parameter labels: {fitter_mm.parameter_labels}")
print(f"\nMixture model parameters per dataset:")
print(f"  P_b: Outlier probability")
print(f"  V_b: Outlier variance")
print(f"  Y_b: Outlier mean")

## 4. Advanced Features <a name="advanced-features"></a>

### 4.1 Multi-Dataset Fitting

Fit data from multiple observatories simultaneously.

In [ ]:
# Generate data from multiple observatories
time1, flux1, err1 = generate_synthetic_event(t0=7500, u0=0.1, tE=30, noise_level=20)
time2, flux2, err2 = generate_synthetic_event(t0=7500, u0=0.1, tE=30, F_source=1500, F_blend=300, noise_level=25)
time3, flux3, err3 = generate_synthetic_event(t0=7500, u0=0.1, tE=30, F_source=800, F_blend=600, noise_level=15, n_points=400)

# Create multi-dataset dictionary
multi_data = {
    'OGLE': (time1, flux1, err1),
    'MOA': (time2, flux2, err2),
    'KMTNet': (time3, flux3, err3)
}

# Visualize all datasets
plt.figure(figsize=(12, 6))
colors = ['red', 'blue', 'green']
for i, (name, (t, f, e)) in enumerate(multi_data.items()):
    plt.errorbar(t, f, yerr=e, fmt='.', alpha=0.5, color=colors[i], label=name)
plt.xlabel('Time (HJD - 2450000)')
plt.ylabel('Flux')
plt.title('Multi-Observatory Microlensing Event')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Initialize fitter with multiple datasets
fitter_multi = SingleLensFitter(
    data=multi_data,
    initial_parameters=initial_params,
    reference_source='OGLE'  # Use OGLE as reference for combined plots
)

print(f"Fitting {len(multi_data)} datasets simultaneously")

### 4.2 Configuring MCMC Settings

Customize the MCMC sampler parameters.

In [ ]:
# Configure MCMC settings
fitter_multi.nwalkers = 50                  # Number of MCMC walkers
fitter_multi.nsteps = 100                   # Steps per burn-in iteration
fitter_multi.nsteps_production = 500        # Steps in production run
fitter_multi.max_burnin_iterations = 50     # Maximum burn-in iterations

# Convergence thresholds
fitter_multi.emcee_lnp_convergence_threshold = 0.5
fitter_multi.emcee_mean_convergence_threshold = 0.1
fitter_multi.emcee_std_convergence_threshold = 0.1

# Output settings
fitter_multi.plotprefix = 'multi_obs_fit'
fitter_multi.make_plots = True
fitter_multi.n_plot_samples = 30            # Number of posterior samples to plot

print("MCMC configuration:")
print(f"  Walkers: {fitter_multi.nwalkers}")
print(f"  Burn-in steps: {fitter_multi.nsteps}")
print(f"  Production steps: {fitter_multi.nsteps_production}")
print(f"  Output prefix: {fitter_multi.plotprefix}")

### 4.3 Setting Parameter Limits

Customize the prior ranges for parameters.

In [ ]:
# Set custom parameter limits (priors)
fitter_multi.u0_limits = (0.0, 1.0)         # Impact parameter range
fitter_multi.t0_limits = (7400, 7600)       # Time of closest approach range
fitter_multi.tE_limits = (10, 100)          # Einstein crossing time range

# For finite source models
fitter_multi.lrho_limits = (-6, 0)          # log10(rho) range

# For Gaussian Process models
fitter_multi.ln_a_limits = (-5, 15)         # GP amplitude range
fitter_multi.ln_tau_limits = (-5.5, 7.0)    # GP timescale range

print("Parameter limits set")

## 5. Complete Example with All Features <a name="complete-example"></a>

This example demonstrates a full analysis pipeline with multiple model components.

In [ ]:
# Create a comprehensive fitter with multiple features
print("Setting up comprehensive model...\n")

fitter_full = SingleLensFitter(
    data=multi_data,
    initial_parameters=np.array([0.12, 7498.0, 28.0]),
    reference_source='OGLE',
    ZP=28.0
)

# Add model complexity
print("Adding model components:")
print("  - Finite source with limb darkening")
fitter_full.add_limb_darkening(gamma=0.5, lrho=-2.5)

print("  - Gaussian Process for correlated noise")
fitter_full.add_gaussian_process_model(common=True)

# Configure MCMC (shorter run for demo)
fitter_full.nwalkers = 50
fitter_full.nsteps = 50
fitter_full.nsteps_production = 200
fitter_full.max_burnin_iterations = 10
fitter_full.plotprefix = 'example_full_fit'
fitter_full.make_plots = True

print(f"\nTotal parameters to fit: {fitter_full.ndim}")
print(f"Parameters: {fitter_full.parameter_labels}")

In [ ]:
# Run the MCMC sampler
print("Running MCMC sampler...\n")
print("This may take several minutes...\n")

# Note: Set optimize_first=True to find a good starting point
fitter_full.sample(optimize_first=True)

print("\n" + "="*50)
print("MCMC SAMPLING COMPLETE")
print("="*50)

In [ ]:
# Display results
print("\nFinal Results:")
print("="*50)
for i, label in enumerate(fitter_full.parameter_labels):
    print(f"{label}: {fitter_full.p[i]:.6f}")

print("\nOutput files generated:")
print(f"  - {fitter_full.plotprefix}.fit_results")
print(f"  - {fitter_full.plotprefix}-combined-lightcurve.png")
print(f"  - {fitter_full.plotprefix}-lc.png")
print(f"  - {fitter_full.plotprefix}-pdist.png")
print(f"  - {fitter_full.plotprefix}-burnin.png")
print(f"  - {fitter_full.plotprefix}-final.png")

In [ ]:
# Display the generated plots
import os
from IPython.display import Image, display

plot_files = [
    f"{fitter_full.plotprefix}-combined-lightcurve.png",
    f"{fitter_full.plotprefix}-pdist.png",
    f"{fitter_full.plotprefix}-final.png"
]

for plot_file in plot_files:
    if os.path.exists(plot_file):
        print(f"\n{plot_file}:")
        display(Image(filename=plot_file))
    else:
        print(f"\n{plot_file}: Not found")

## 6. Template for Your Own Data <a name="your-data-template"></a>

Use this template to analyze your own microlensing data. Simply modify the data loading section to read your files.

In [ ]:
# ============================================================================
# TEMPLATE: Analyze Your Own Microlensing Data
# ============================================================================

# STEP 1: Load your data
# Replace this section with code to load your actual data files
# Your data should have columns: time, flux, flux_error

def load_your_data():
    """
    Load microlensing data from your files.
    
    Returns:
    --------
    data_dict : dict
        Dictionary with observatory names as keys and (time, flux, error) tuples as values
    """
    
    # Example: Loading from text files
    # Uncomment and modify for your data format
    
    # Option 1: Single file with multiple columns
    # data = np.loadtxt('my_lightcurve.dat')
    # time, flux, flux_err = data[:, 0], data[:, 1], data[:, 2]
    # data_dict = {'MyTelescope': (time, flux, flux_err)}
    
    # Option 2: Multiple files from different observatories
    # ogle_data = np.loadtxt('ogle_data.txt')
    # moa_data = np.loadtxt('moa_data.txt')
    # data_dict = {
    #     'OGLE': (ogle_data[:, 0], ogle_data[:, 1], ogle_data[:, 2]),
    #     'MOA': (moa_data[:, 0], moa_data[:, 1], moa_data[:, 2])
    # }
    
    # Option 3: CSV files with pandas
    # import pandas as pd
    # df = pd.read_csv('my_data.csv')
    # data_dict = {'Survey': (df['time'].values, df['flux'].values, df['error'].values)}
    
    # For this template, we'll use synthetic data
    print("Using synthetic data for demonstration.")
    print("Replace this function with your actual data loading code.\n")
    
    time, flux, err = generate_synthetic_event(
        t0=7500.0, u0=0.1, tE=30.0,
        F_source=1000.0, F_blend=500.0,
        noise_level=20.0, n_points=500
    )
    
    return {'YourObservatory': (time, flux, err)}

# Load data
your_data = load_your_data()
print(f"Loaded data from {len(your_data)} observatory/observatories")
for name, (t, f, e) in your_data.items():
    print(f"  {name}: {len(t)} data points")

In [ ]:
# STEP 2: Set initial parameter estimates
# Examine your light curve and estimate:
#   u0: Impact parameter (typically 0.01 - 1.0)
#   t0: Time of peak magnification (inspect your light curve)
#   tE: Duration of event in days (full width ~ 2*tE)

initial_u0 = 0.1      # Impact parameter estimate
initial_t0 = 7500.0   # Time of peak (HJD - 2450000)
initial_tE = 30.0     # Event duration estimate

your_initial_params = np.array([initial_u0, initial_t0, initial_tE])

print(f"Initial parameter guesses:")
print(f"  u0 = {initial_u0}")
print(f"  t0 = {initial_t0}")
print(f"  tE = {initial_tE}")

In [ ]:
# STEP 3: Initialize the fitter
your_fitter = SingleLensFitter(
    data=your_data,
    initial_parameters=your_initial_params,
    reference_source=list(your_data.keys())[0],  # First observatory as reference
    ZP=28.0  # Adjust if needed for your photometric system
)

print(f"Fitter initialized with {your_fitter.ndim} parameters")

In [ ]:
# STEP 4: Add model components (customize based on your needs)

# Uncomment the features you want to include:

# Add finite source effects (for high magnification events)
# your_fitter.add_finite_source(lrho=-3.0)

# Add limb darkening (includes finite source)
# your_fitter.add_limb_darkening(gamma=0.5, lrho=-2.5)

# Add source variability
# your_fitter.add_source_variability(params=(0.001, np.pi, 0.0))

# Add blend variability
# your_fitter.add_blend_variability(params=(0.001, np.pi, 0.0))

# Add Gaussian Process for correlated noise
# your_fitter.add_gaussian_process_model(common=True)

# Add mixture model for outlier rejection
# your_fitter.add_mixture_model()

print(f"Model configured with {your_fitter.ndim} parameters")
print(f"Parameters: {your_fitter.parameter_labels}")

In [ ]:
# STEP 5: Configure MCMC settings

# For quick testing (low accuracy)
your_fitter.nwalkers = 50
your_fitter.nsteps = 100
your_fitter.nsteps_production = 500
your_fitter.max_burnin_iterations = 20

# For publication-quality results, increase these values:
# your_fitter.nwalkers = 100
# your_fitter.nsteps = 200
# your_fitter.nsteps_production = 2000
# your_fitter.max_burnin_iterations = 100

# Set output file prefix
your_fitter.plotprefix = 'my_event_fit'
your_fitter.make_plots = True

print("MCMC settings configured")

In [ ]:
# STEP 6: Set parameter limits (optional but recommended)

# Adjust these based on your event
your_fitter.u0_limits = (0.0, 1.5)           # Impact parameter range
your_fitter.t0_limits = (initial_t0 - 100, initial_t0 + 100)  # t0 ± 100 days
your_fitter.tE_limits = (5, 200)             # Einstein time range

# For finite source models
your_fitter.lrho_limits = (-6, 0)

# For GP models
your_fitter.ln_a_limits = (-5, 15)
your_fitter.ln_tau_limits = (-5.5, 7.0)

print("Parameter limits set")

In [ ]:
# STEP 7: Run optimization first (recommended)

print("Running optimization to find good starting point...\n")
your_fitter.fit(method='Nelder-Mead')

print("\nOptimized parameters:")
for i, label in enumerate(your_fitter.parameter_labels):
    print(f"  {label}: {your_fitter.p[i]:.6f}")

In [ ]:
# STEP 8: Run MCMC sampler

print("Starting MCMC sampling...")
print("This may take several minutes to hours depending on your settings.\n")

# Run the sampler (optimize_first=False since we already optimized)
your_fitter.sample(optimize_first=False)

print("\n" + "="*70)
print("SAMPLING COMPLETE!")
print("="*70)

In [ ]:
# STEP 9: Analyze results

print("\nFinal Best-Fit Parameters (Median of Posterior):")
print("="*70)
for i, label in enumerate(your_fitter.parameter_labels):
    print(f"{label}: {your_fitter.p[i]:.6f}")

print("\n\nOutput Files:")
print("="*70)
output_files = [
    f"{your_fitter.plotprefix}.fit_results (parameter table)",
    f"{your_fitter.plotprefix}-combined-lightcurve.png (combined light curve)",
    f"{your_fitter.plotprefix}-lc.png (individual light curves)",
    f"{your_fitter.plotprefix}-pdist.png (corner plot of posteriors)",
    f"{your_fitter.plotprefix}-burnin.png (burn-in chains)",
    f"{your_fitter.plotprefix}-final.png (production chains)",
    f"{your_fitter.plotprefix}-state-production.npy (final MCMC state)"
]
for f in output_files:
    print(f"  - {f}")

print("\n" + "="*70)
print("Analysis complete! Check the output files for detailed results.")
print("="*70)

In [ ]:
# STEP 10: Display generated plots

plot_files = [
    (f"{your_fitter.plotprefix}-combined-lightcurve.png", "Combined Light Curve"),
    (f"{your_fitter.plotprefix}-pdist.png", "Posterior Distributions"),
    (f"{your_fitter.plotprefix}-final.png", "MCMC Chains")
]

for filename, title in plot_files:
    if os.path.exists(filename):
        print(f"\n{title}:")
        print("="*70)
        display(Image(filename=filename))
    else:
        print(f"\n{title}: File not found ({filename})")

## Additional Tips and Best Practices

### Choosing Initial Parameters
- **u0**: Look at the peak magnification. Higher peaks → smaller u0
- **t0**: The time at maximum flux
- **tE**: Roughly half the FWHM of the light curve

### Model Selection
- Start with the simplest PSPL model
- Add complexity only if needed (high magnification, visible deviations, etc.)
- Use Gaussian Processes if you see correlated residuals
- Use mixture models if you have obvious outliers

### MCMC Settings
- **Testing**: Use `nwalkers=50`, `nsteps_production=500`
- **Production**: Use `nwalkers=100-200`, `nsteps_production=2000-5000`
- Always run `optimize_first=True` for the first run
- Check convergence plots (`*-final.png`) to ensure chains have converged

### Interpreting Results
- Check the corner plot (`*-pdist.png`) for parameter correlations
- Examine residuals in the combined light curve plot
- Review the MCMC chains to ensure they've stabilized
- The `.fit_results` file contains the 50th percentile and uncertainties

### Troubleshooting
- **Poor convergence**: Increase `nsteps` or `max_burnin_iterations`
- **Bad fit**: Check initial parameters, try optimization first
- **Slow performance**: Reduce `nwalkers` or `nsteps_production` for testing
- **Unrealistic parameters**: Adjust parameter limits

---

## References

If you use this code, please cite:

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.265134.svg)](https://doi.org/10.5281/zenodo.265134)

For more information, see the [GitHub repository](https://github.com/your-username/SingleLensFitter) and the wiki.

## Decision Guide: Which Features Should I Use?

### For Quick Testing and Exploration
```python
fitter.use_fast_linear_fit = True  # 2-5× faster
# Use simple PSPL model only
# Reduce nwalkers and nsteps_production
```

### For Data with Systematic Trends
```python
# Create eigen lightcurves from reference stars or PCA
eigen_lcs = {'Observatory': eigen_array}
fitter = SingleLensFitter(data=data, initial_parameters=params, 
                          eigen_lightcurves=eigen_lcs)
```

### For High-Magnification Events
```python
fitter.add_finite_source(lrho=-3.0)  # When u0 < 0.1
# or
fitter.add_limb_darkening(gamma=0.5, lrho=-2.5)  # For very high mag
```

### For Correlated Noise
```python
fitter.add_gaussian_process_model(common=True)
# Use when residuals show time-correlated structure
```

### For Data with Outliers
```python
fitter.add_mixture_model()
# Robust fitting that downweights outliers
```

### For Variable Stars
```python
fitter.add_source_variability(params=(0.01, np.pi, 0.0))
# or
fitter.add_blend_variability(params=(0.01, np.pi, 0.0))
```

### Example: Production-Quality Fit
```python
# Comprehensive model for publication
fitter = SingleLensFitter(data=multi_obs_data, 
                          initial_parameters=params,
                          eigen_lightcurves=eigen_lcs,
                          reference_source='OGLE')
fitter.add_limb_darkening(gamma=0.5, lrho=-2.5)
fitter.add_gaussian_process_model(common=True)
fitter.nwalkers = 100
fitter.nsteps_production = 2000
fitter.max_burnin_iterations = 50
fitter.sample(optimize_first=True)
```

## Feature Compatibility Matrix

Use this table to understand which features can be used together:

| Feature | use_fast_linear_fit | eigen_lightcurves | GP | Mixture | Variability | Finite Source | Limb Darkening |
|---------|---------------------|-------------------|----|---------|-----------|--------------|--------------  |
| **use_fast_linear_fit** | ✓ | ❌ | ❌ | ❌ | ❌ | ✓ | ✓ |
| **eigen_lightcurves** | ❌ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Gaussian Process** | ❌ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Mixture Model** | ❌ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Source/Blend Variability** | ❌ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Finite Source** | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Limb Darkening** | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ (auto) | ✓ |

**Key Points**:
- `use_fast_linear_fit` is **only** compatible with simple PSPL models (with optional finite source/limb darkening)
- `eigen_lightcurves` is compatible with everything **except** `use_fast_linear_fit`
- All other features are mutually compatible
- Limb darkening automatically includes finite source effects